In [4]:
import requests
import pymongo
import pandas as pd
import numpy
import datetime
import os
import json

def batched(iterable, batch_size=10):
    """
    Sequentially yield segments of the iterable in lists of the given size.
    """
    batch = []
    for idx, item in enumerate(iterable):
        batch.append(item)
        batch_idx = idx % batch_size
        if batch_idx == batch_size - 1:
            yield batch
            batch = []
    yield batch

db = pymongo.MongoClient(os.environ['MONGO_HOST'])['flirt']

"Notebook evaluated on: " + str(datetime.datetime.now())

'Notebook evaluated on: 2018-02-24 00:18:11.598106'

In [5]:
events = requests.get('https://eidr-connect.eha.io/api/auto-events', params={
    'limit': 20000
}).json()
len(events)

221

In [6]:
passenger_flows = list(db.passengerFlows.find({
    'simGroup': 'ibis14day'
}))
airport_set = set()
for flow in passenger_flows:
    airport_set.add(flow['arrivalAirport'])
    airport_set.add(flow['departureAirport'])
airport_to_idx = {}
airports = []
for idx, airport in enumerate(airport_set):
    airport_to_idx[airport] = idx
    airports.append(airport)
len(airports)

3806

In [8]:
flow_matrix = numpy.zeros(shape=(len(airport_set), len(airport_set)))
for flow in passenger_flows:
    # Remove US airports from departures?
    flow_matrix[
        airport_to_idx[flow['departureAirport']],
        airport_to_idx[flow['arrivalAirport']]] = flow['estimatedPassengers']
flow_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
resolved_events = []
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=14)

def resolved_event_iter(events):
    for event_batch in batched(events, 5):
        results = requests.get('https://eidr-connect.eha.io/api/events-with-resolved-data', params={
            'ids': [event['_id'] for event in event_batch],
            'startDate': start_date.isoformat(),
            'endDate': end_date.isoformat(),
            'eventType': 'auto'
        }).json()['events']
        for result in results:
            yield result

events_with_resolved_data = list(zip(events, resolved_event_iter(events)))


In [10]:
# Initialize with names used by airport data set that are not present in the
# world geojson file.
nameToISOs = {
  'North Korea': 'KP',
  'South Korea': 'KR',
  'United States Minor Outlying Islands': 'US',
  'Macau': 'MO',
  'Reunion': 'RE',
  'Christmas Island': 'CX',
  'Guadeloupe': 'GP',
  'Ivory Coast (Cote d\'Ivoire)': 'CI',
  'French Guiana': 'GF',
  'Western Samoa': 'WS',
  'Saint Vincent and Grenadines': 'VC',
  'Guinea Bissau': 'GW',
  'Cocos (Keeling) Islands': 'CC',
  'Grenada and South Grenadines': 'GD',
  'Mayotte': 'YT',
  'Martinique': 'MQ',
  'Tuvalu': 'TV',
  'Gibraltar': 'GI',
  'Bonaire, Saint Eustatius & Saba': None,
  'Palestinian Territory': None,
  'Curacao': None,
  'Unknown Country': None
}
name_props = [
  'name',
  'name_long',
  'formal_en',
  'name_alt',
  'name_sort',
  'formal_en',
  'brk_name'
]


with open("../imports/geoJSON/world.geo.json") as f:
    world_geo_json = json.load(f)
    for feature in world_geo_json['features']:
        properties = feature['properties']
        for prop in name_props:
            value = properties.get(prop)
            if value:
                nameToISOs[value] = properties['iso_a2']

    airport_to_country_code = {
        airport['_id']: nameToISOs[airport['countryName']]
        for airport in db.airports.find({})
    }

    countries_by_code = {
        feature['properties']['iso_a2']: feature['properties']
        for feature in world_geo_json['features']
        if feature['properties']['iso_a2']
    }


In [11]:
probabilty_passenger_infected_matrix = numpy.zeros(shape=(len(events), len(airport_set)))       
for idx, (event, resolved_event) in enumerate(events_with_resolved_data):
    for airport in airports:
        cc = airport_to_country_code.get(airport)
        country_data = countries_by_code.get(cc)
        if country_data:
            prob = float(resolved_event['locations'].get(cc, 0)) / country_data['pop_est']
        else:
            prob = 0
        probabilty_passenger_infected_matrix[idx, airport_to_idx[airport]] = prob
probabilty_passenger_infected_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
for idx, (event, resolved_event) in enumerate(events_with_resolved_data):
    db.resolvedEvents_create.insert_one(dict(
        resolved_event,
        _id=event['_id'],
        name=event['eventName'],
        timestamp=datetime.datetime.now()))
db.resolvedEvents_create.rename("resolvedEvents", dropTarget=True)

{u'ok': 1.0}

In [13]:
# Determine DALYs Per case using GBD data.
# Citation:
# Global Burden of Disease Collaborative Network.
# Global Burden of Disease Study 2016 (GBD 2016) Results.
# Seattle, United States: Institute for Health Metrics and Evaluation (IHME), 2017.
# Available from http://ghdx.healthdata.org/gbd-results-tool?params=gbd-api-2016-permalink/4ef232ed16d847a5cad84132845b2af7
import epitator
from epitator.database_interface import DatabaseInterface
epitator_db = DatabaseInterface()
df = pd.read_csv("IHME-GBD_2016_DATA.csv")
formatted_df = df[df.year > 2000]\
    .groupby(['cause', 'measure']).mean().reset_index()\
    .pivot(index='cause', columns='measure', values='val').reset_index()
# Incidence rather than prevalance is used because it, like DALYs, gives a annual rate.
# The DALYs for a given year can come from cases that occurred in previous years,
# and the temporal offset is dependent on the disease. To reduce the effects
# of this descrepancy the annual rates are averaged over the a 10-20 year period.
formatted_df['ratio'] = formatted_df['DALYs (Disability-Adjusted Life Years)'] / formatted_df['Incidence']
formatted_df.sort_values('ratio')

measure,cause,DALYs (Disability-Adjusted Life Years),Incidence,Prevalence,ratio
64,Upper respiratory infections,5.650730e+06,1.664982e+10,2.205579e+08,0.000339
69,Zika virus,3.239529e+02,4.748786e+05,8.037193e+03,0.000682
60,Trichomoniasis,1.773682e+05,1.343113e+08,1.543474e+08,0.001321
0,Acute hepatitis A,7.448615e+05,2.767372e+08,2.128748e+07,0.002692
20,Genital herpes,1.951324e+05,5.370999e+07,7.571730e+08,0.003633
21,Gonococcal infection,6.046669e+05,1.656256e+08,4.966329e+07,0.003651
65,Varicella and herpes zoster,1.016325e+06,1.402533e+08,6.804966e+06,0.007246
51,Otitis media,3.177384e+06,4.288391e+08,1.030776e+08,0.007409
5,Chlamydial infection,5.307186e+05,7.115112e+07,8.910134e+07,0.007459
29,Hepatitis C,8.969458e+04,6.884205e+06,1.351504e+08,0.013029


In [14]:
# Get total DALYs per case for top level disease categories
totals = formatted_df[formatted_df.cause.isin([
    'Sexually transmitted diseases excluding HIV',
    'HIV/AIDS and tuberculosis',
    'Diarrhea, lower respiratory, and other common infectious diseases',
    'Neglected tropical diseases and malaria'])].sum()
average_DALYs_per_case = totals['DALYs (Disability-Adjusted Life Years)'] / totals['Incidence']
average_DALYs_per_case

0.02449616228342763

In [15]:
def valid_ratio(x):
    return not(pd.isna(x) or x > 100)

name_mappings = {
    'Acute hepatitis A': 'hepatitis A',
    'Acute hepatitis E': 'hepatitis E',
    'Cutaneous and mucocutaneous leishmaniasis': 'leishmaniasis',
    'Diarrheal diseases': 'diarrhea',
    'Food-borne trematodiases': 'trematodiases',
    'HIV/AIDS': 'hiv',
    'Hookworm disease': 'Hookworm',
    'Latent tuberculosis infection': 'tuberculosis',
    'Varicella and herpes zoster': 'Varicella',
}

disease_uri_to_DALYs_per_case = {}
for idx, x in formatted_df.iterrows():
    cause = x['cause']
    if cause in name_mappings:
        cause = name_mappings[cause]
    disease_ent = next(epitator_db.lookup_synonym(cause, 'disease'), None)
    if disease_ent:
        disease_uri_to_DALYs_per_case[disease_ent['id']] = disease_uri_to_DALYs_per_case.get(disease_ent['id'], []) + [{
            'label': disease_ent['label'],
            'DALYsPerCase': x['ratio'] if valid_ratio(x['ratio']) else average_DALYs_per_case,
            'weight': disease_ent['weight']
        }]
    else:
        print("Unresolved: " + x['cause'])
disease_uri_to_DALYs_per_case = {
    k: max(v, key=lambda x: x['weight'])['DALYsPerCase']
    for k, v in disease_uri_to_DALYs_per_case.items()
}

Unresolved: Diarrhea, lower respiratory, and other common infectious diseases
Unresolved: Diarrheal diseases
Unresolved: Drug-susceptible HIV/AIDS - Tuberculosis
Unresolved: Drug-susceptible tuberculosis
Unresolved: Extensively drug-resistant HIV/AIDS - Tuberculosis
Unresolved: Extensively drug-resistant tuberculosis
Unresolved: Food-borne trematodiases
Unresolved: Gonococcal infection
Unresolved: Guinea worm disease
Unresolved: H influenzae type B meningitis
Unresolved: HIV/AIDS and tuberculosis
Unresolved: HIV/AIDS resulting in other diseases
Unresolved: Hookworm disease
Unresolved: Intestinal infectious diseases
Unresolved: Intestinal nematode infections
Unresolved: Lower respiratory infections
Unresolved: Meningococcal meningitis
Unresolved: Multidrug-resistant HIV/AIDS - Tuberculosis without extensive drug resistance
Unresolved: Multidrug-resistant tuberculosis without extensive drug resistance
Unresolved: Neglected tropical diseases and malaria
Unresolved: Other infectious diseas

In [16]:
def gen_ranks():
    for idx, (event, resolved_event) in enumerate(events_with_resolved_data):
        event_id = event['_id']
        DALYs_per_case = disease_uri_to_DALYs_per_case.get(event['diseases'][0]['id'], average_DALYs_per_case)
        for arrival_airport, country_code in airport_to_country_code.items():
            if country_code != "US" or arrival_airport not in airport_to_idx:
                continue
            arrival_airport_idx = airport_to_idx[arrival_airport]
            for departure_airport, departure_country_code in airport_to_country_code.items():
                if departure_country_code not in resolved_event['locations'] or departure_airport not in airport_to_idx:
                    continue
                dep_airport_idx = airport_to_idx[departure_airport]
                rank_score = probabilty_passenger_infected_matrix[idx, dep_airport_idx] * flow_matrix[
                        dep_airport_idx, arrival_airport_idx]
                if rank_score == 0:
                    continue
                yield {
                    'eventId': event_id,
                    'departureAirportId': departure_airport,
                    'arrivalAirportId': arrival_airport,
                    'probabilityPassengerInfected': probabilty_passenger_infected_matrix[idx, dep_airport_idx],
                    'passengerFlow': flow_matrix[dep_airport_idx, arrival_airport_idx],
                    'threatCoefficient': DALYs_per_case,
                    'rank': rank_score * DALYs_per_case
                }

for ranks in batched(gen_ranks(), 50000):
    result = db.eventAirportRanks_create.insert_many(ranks)
    print(len(result.inserted_ids), '/', len(ranks), 'records inserted')
db.eventAirportRanks_create.rename("eventAirportRanks", dropTarget=True)

(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(50000, '/', 50000, 'records inserted')
(28233, '/', 28233, 'records inserted')


{u'ok': 1.0}

In [17]:
db.eventAirportRanks.find_one({
    'rank': {'$gt': 0}
})

{u'_id': ObjectId('5a90b52633919a40066e0093'),
 u'arrivalAirportId': u'SGY',
 u'departureAirportId': u'PSG',
 u'eventId': u'zPKWCnRYNK85hAKig',
 u'rank': 3.3830205556694816e-07}